In [24]:
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
import pycolmap
from deep_image_matching.utils import COLMAPDatabase

root_path = Path("datasets/belv_20230725")
image_dir = root_path / "images"

sfm_dir = root_path / "results_superpoint+lightglue_bruteforce_quality_highest"
sfm_rec_path = sfm_dir / "reconstruction"

output_path = root_path / "marker_triang"
db_path = output_path / "database_markers.db"

markers_file = root_path / "markers_image_20230725.csv"

output_path.mkdir(exist_ok=True, parents=True)

# Get image list
images = sorted(image_dir.glob("*"))

In [25]:
# Read marker image coordinates and create keypoints dictionary with the form:
# {"image_name": keypoints_array}
# {
#     "image1.jpg": np.array([[x1, y1], [x2, y2], ...]),
#     "image2.jpg": np.array([[x1, y1], [x2, y2], ...]),
#     ...
# }
markers_image = pd.read_csv(
    markers_file, header=None, names=["image", "marker", "x", "y"]
)
markers_image.sort_values(["image", "marker"], inplace=True, ascending=True)
kpts = {}
for image, gr in markers_image.groupby("image"):
    image = image + ".JPG"
    kpts[image] = gr[["x", "y"]].values
for k, v in kpts.items():
    print(k, ":\n", v)

# Manually create 1-to-1 matches array
ids = np.arange(0, len(kpts[images[0].name]))
matches_idx = np.array([ids, ids]).astype(np.int64).T
print("matches idx:\n", matches_idx)

p1_20230725_115953_IMG_1147.JPG :
 [[4832.3403 1582.9143]
 [2027.8405 3478.0388]
 [ 725.7628 3845.6572]
 [4092.3181 2256.6533]
 [3416.7617 3300.4646]
 [5580.9126  552.5623]]
p2_20230725_120026_IMG_0885.JPG :
 [[3194.6826 1729.3799]
 [3573.5344 3204.1846]
 [4105.1738 3556.3171]
 [4139.4536 1980.674 ]
 [5928.5713 2516.8994]
 [1994.9277 1340.9641]]
matches idx:
 [[0 0]
 [1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]]


In [26]:
# # plot image with markers
# import cv2
# from matplotlib import pyplot as plt

# image_id = 1

# img = cv2.cvtColor(cv2.imread(str(images[image_id])), cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# plt.scatter(
#     kpts[images[image_id].name][:, 0],
#     kpts[images[image_id].name][:, 1],
#     c="r",
#     s=10,
# )

In [27]:
# Save features to h5 file
features_h5 = output_path / "features.h5"
with h5py.File(features_h5, "w") as f:
    for image in images:
        image_name = image.name
        kp = kpts[image_name]
        f.create_group(image_name)
        f[image_name].create_dataset("keypoints", data=kp, dtype=np.float32)

# Save matches to h5 file
matches_h5 = output_path / "matches.h5"
with h5py.File(matches_h5, "w") as f:
    image0, image1 = images[0].name, images[1].name
    gr0 = f.create_group(image0)
    gr0.create_dataset(image1, data=matches_idx, dtype=np.int64)

pair_file = sfm_dir / "pairs.txt"

# # print features_h5 content
# with h5py.File(features_h5, "r") as f:
#     print(f.keys())
#     print(f[images[0].name].keys())
#     print(f[images[0].name]["keypoints"][:])
#     print(f[images[1].name].keys())
#     print(f[images[1].name]["keypoints"][:])

# # Print matches.h5 content
# with h5py.File(matches_h5, "r") as f:
#     print(f.keys())
#     g0 = f[images[0].name]
#     print(g0.keys())
#     g1 = g0[images[1].name]
#     print(g1.__array__())

In [28]:
import pycolmap
from deep_image_matching.triangulation import db_from_existing_poses
from deep_image_matching.utils import OutputCapture

sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create a new database with the dense features and the known camera poses

db_from_existing_poses(
    db_path,
    features_h5,
    matches_h5,
    sfm_rec_path,
    pair_file,
    do_geometric_verification=False,
)

2024-04-10 16:59:39 | [WARNING ] The database already exists, deleting it.


Importing matches: 100%|██████████| 1/1 [00:00<00:00, 1518.03it/s]


In [41]:
# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    min_num_matches=3,
    triangulation=dict(
        ignore_two_view_tracks=False,
    ),
)

IncrementalPipelineOptions:
    min_num_matches = 15
    ignore_watermarks = False
    multiple_models = True
    max_num_models = 50
    max_model_overlap = 20
    min_model_size = 10
    init_image_id1 = -1
    init_image_id2 = -1
    init_num_trials = 200
    extract_colors = True
    num_threads = -1
    min_focal_length_ratio = 0.1
    max_focal_length_ratio = 10.0
    max_extra_param = 1.0
    ba_refine_focal_length = True
    ba_refine_principal_point = False
    ba_refine_extra_params = True
    ba_min_num_residuals_for_multi_threading = 50000
    ba_local_num_images = 6
    ba_local_function_tolerance = 0.0
    ba_local_max_num_iterations = 25
    ba_global_images_ratio = 1.1
    ba_global_points_ratio = 1.1
    ba_global_images_freq = 500
    ba_global_points_freq = 250000
    ba_global_function_tolerance = 0.0
    ba_global_max_num_iterations = 50
    ba_local_max_refinements = 2
    ba_local_max_refinement_change = 0.001
    ba_global_max_refinements = 5
    ba_global_max_r

In [42]:
# Run the triangulation with the known camera poses
verbose = True
with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            db_path,
            image_dir,
            output_path,
            options=opt,
        )

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.014301e+00    0.00e+00    1.32e+03   0.00e+00   0.00e+00  1.00e+04        0    2.60e-05    5.70e-05
   1  2.637796e+00    3.77e-01    1.42e-01   6.52e-04   1.00e+00  3.00e+04        0    4.10e-05    8.31e-04


I20240410 17:06:20.199561 1082195 misc.cc:198] 
Loading database
I20240410 17:06:20.200467 1082195 database_cache.cc:54] Loading cameras...
I20240410 17:06:20.200500 1082195 database_cache.cc:64]  2 in 0.000s
I20240410 17:06:20.200510 1082195 database_cache.cc:72] Loading matches...
I20240410 17:06:20.200522 1082195 database_cache.cc:78]  1 in 0.000s
I20240410 17:06:20.200526 1082195 database_cache.cc:94] Loading images...
I20240410 17:06:20.200562 1082195 database_cache.cc:143]  2 in 0.000s (connected 2)
I20240410 17:06:20.200567 1082195 database_cache.cc:154] Building correspondence graph...
I20240410 17:06:20.200574 1082195 database_cache.cc:190]  in 0.000s (ignored 0)
I20240410 17:06:20.200598 1082195 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240410 17:06:20.200753 1082195 misc.cc:198] 
Triangulating image #1 (0)
I20240410 17:06:20.200776 1082195 sfm.cc:473] => Image sees 0 / 6 points
I20240410 17:06:20.200824 1082195 sfm.cc:478] => Triangulated 6 points
I20240410 17:06:20.2008

In [43]:
pts = reconstruction.points3D
for i, pt in pts.items():
    print(i, pt)

1507 Point3D(xyz=[11.4798, -5.21418, 25.8772], color=[100, 108, 96], error=0.405386, track=Track(length=2))
1506 Point3D(xyz=[3.78636, 0.790003, 6.84661], color=[254, 243, 251], error=0.143108, track=Track(length=2))
1505 Point3D(xyz=[4.91828, -0.280677, 10.1311], color=[234, 242, 242], error=0.00433232, track=Track(length=2))
1504 Point3D(xyz=[0.691425, 1.45129, 6.99773], color=[244, 228, 229], error=1.49442, track=Track(length=2))
1503 Point3D(xyz=[1.99842, 1.31624, 8.79684], color=[163, 107, 101], error=0.358146, track=Track(length=2))
1502 Point3D(xyz=[6.80372, -1.44258, 14.1866], color=[143, 139, 86], error=0.0227758, track=Track(length=2))


In [44]:
img = reconstruction.images[1]
img.points2D

[Point2D(xy=[4832.84, 1583.41], point3D_id=1502),
 Point2D(xy=[2028.34, 3478.54], point3D_id=1503),
 Point2D(xy=[726.263, 3846.16], point3D_id=1504),
 Point2D(xy=[4092.82, 2257.15], point3D_id=1505),
 Point2D(xy=[3417.26, 3300.96], point3D_id=1506),
 Point2D(xy=[5581.41, 553.062], point3D_id=1507)]

In [45]:
markers_image

,image,marker,x,y
0,p1_20230725_115953_IMG_1147,D38,4832.3403,1582.9143
4,p1_20230725_115953_IMG_1147,F10,2027.8405,3478.0388
2,p1_20230725_115953_IMG_1147,F2,725.7628,3845.6572
5,p1_20230725_115953_IMG_1147,F20,4092.3181,2256.6533
3,p1_20230725_115953_IMG_1147,F4,3416.7617,3300.4646
1,p1_20230725_115953_IMG_1147,T2,5580.9126,552.5623
6,p2_20230725_120026_IMG_0885,D38,3194.6826,1729.3799
10,p2_20230725_120026_IMG_0885,F10,3573.5344,3204.1846
8,p2_20230725_120026_IMG_0885,F2,4105.1738,3556.3171
11,p2_20230725_120026_IMG_0885,F20,4139.4536,1980.6740
